In [7]:
import os
import re
import subprocess

1. Clone git repo.

In [4]:
def clone_repo(github_repo):
    repo_name = github_repo.split("/")[-1].replace(".git", "")
    repo_path = f"data/{repo_name}"
    print(f"Cloning {github_repo} to {repo_path}")
    if not os.path.exists("data"):
        os.makedirs("data")

    if not os.path.exists(repo_path):
        subprocess.run(["git", "clone", github_repo, repo_path], check=True)
    else:
        print(f"Repository {repo_name} already exists. Skipping clone.")
    
    return repo_name

clone_repo("git@github.com:heysarver/double-entry-accounting.git")

Cloning git@github.com:heysarver/double-entry-accounting.git to data/double-entry-accounting


Cloning into 'data/double-entry-accounting'...


'double-entry-accounting'

2. Search for all files based on a list of file extensions (python only for now).

In [5]:
def get_file_list(repo_name, file_extensions):
    embed_files = []
    file_extensions = file_extensions.split(',')
    for root, dirs, files in os.walk(f"data/{repo_name}"):
        for file in files:
            if any(file.endswith(ext) for ext in file_extensions):
                embed_files.append(os.path.join(root, file))
    return embed_files

files = get_file_list("double-entry-accounting", "py")
print(files)


['data/double-entry-accounting/migrate.py', 'data/double-entry-accounting/run.py', 'data/double-entry-accounting/delete_test_data.py', 'data/double-entry-accounting/import_test_data.py', 'data/double-entry-accounting/migrations/env.py', 'data/double-entry-accounting/migrations/versions/202309071641-9cb1089a9def_add_ondelete_cascade.py', 'data/double-entry-accounting/migrations/versions/202309071654-4ab13a0bd1aa_allow_custom_txnids.py', 'data/double-entry-accounting/migrations/versions/202309071705-194466430716_add_views.py', 'data/double-entry-accounting/migrations/versions/202309071607-99d831242adc_initial_db.py', 'data/double-entry-accounting/app/__init__.py', 'data/double-entry-accounting/app/models.py', 'data/double-entry-accounting/app/config.py', 'data/double-entry-accounting/app/routes/transactions.py', 'data/double-entry-accounting/app/routes/accounts.py', 'data/double-entry-accounting/app/routes/utils.py', 'data/double-entry-accounting/app/seeds/__init__.py', 'data/double-entr

3. Read each file and extract a list of all functions and variables outside of functions.

In [28]:
def extract_python_functions_and_classes(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    functions = []
    classes = []
    current_block = []
    in_function = False
    in_class = False

    for line in lines:
        if re.match(r'^def ', line):
            functions.append(''.join(current_block))
            current_block = [line]
            in_function = True
            in_class = False
        elif re.match(r'^class ', line):
            classes.append(''.join(current_block))
            current_block = [line]
            in_function = False
            in_class = True
        elif re.match(r'^\S', line):
            if in_function:
                functions.append(''.join(current_block))
                current_block = []
                in_function = False
            elif in_class:
                classes.append(''.join(current_block))
                current_block = []
                in_class = False
        elif in_function or in_class:
            current_block.append(line)

    if current_block:
        if in_function:
            functions.append(''.join(current_block))
        elif in_class:
            classes.append(''.join(current_block))

    return functions, classes

print("Functions:")
for f in functions:
    print(f)

print("Classes:")
for c in classes:
    print(c)

Functions:
def get_engine():
    try:
        # this works with Flask-SQLAlchemy<3 and Alchemical
        return current_app.extensions['migrate'].db.get_engine()
    except TypeError:
        # this works with Flask-SQLAlchemy>=3
        return current_app.extensions['migrate'].db.engine



def get_engine_url():
    try:
        return get_engine().url.render_as_string(hide_password=False).replace(
            '%', '%%')
    except AttributeError:
        return str(get_engine().url).replace('%', '%%')



def get_metadata():
    if hasattr(target_db, 'metadatas'):
        return target_db.metadatas[None]
    return target_db.metadata



def run_migrations_offline():
    """Run migrations in 'offline' mode.

    This configures the context with just a URL
    and not an Engine, though an Engine is acceptable
    here as well.  By skipping the Engine creation
    we don't even need a DBAPI to be available.

    Calls to context.execute() here emit the given string to the
    script outp

4. For each function, generate a short description using a code-trained model from huggingface.

In [ ]:
# 4. For each function, generate a short description using a code-trained model from huggingface.

# 5. For each class, generate a short description using a code-trained model from huggingface.
